# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,anadyr,64.7500,177.4833,-20.56,76,0,8.00,RU,1703904235
1,1,udachny,66.4167,112.4000,-25.31,83,100,3.35,RU,1703904236
2,2,whitehorse,60.7161,-135.0538,-10.50,89,100,0.00,CA,1703904153
3,3,grytviken,-54.2811,-36.5092,-0.45,76,84,2.88,GS,1703904236
4,4,bilibino,68.0546,166.4372,-41.80,99,55,1.30,RU,1703904236


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
city_data_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City",
    hover_cols = ["Country", "Max Temp"]
)

# Display the map
city_data_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Max Temp)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
best_city_df = city_data_df.loc[(city_data_df["Max Temp"] < 28) & (city_data_df["Max Temp"] > 21) 
                                & (city_data_df["Wind Speed"] < 6) & (city_data_df["Cloudiness"] == 0),:]

# Drop any rows with null values
best_city_df =best_city_df.dropna()

# Display sample data
best_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
86,86,saint croix,17.7397,-64.7388,23.31,77,0,2.06,VI,1703904259
142,142,bubaque,11.2833,-15.8333,25.87,73,0,4.26,GW,1703904277
169,169,cabo san lucas,22.8909,-109.9124,22.17,65,0,1.03,MX,1703903613
217,217,freetown,8.4840,-13.2299,24.67,82,0,2.66,SL,1703904217
257,257,lazaro cardenas,17.9583,-102.2000,22.53,76,0,1.45,MX,1703904307
260,260,acapulco de juarez,16.8634,-99.8901,24.90,73,0,1.54,MX,1703904308
345,345,praya,14.9215,-23.5087,24.30,35,0,5.66,CV,1703904336
364,364,salaga,8.5508,-0.5188,24.47,18,0,2.37,GH,1703904342
385,385,natal,-5.7950,-35.2094,26.12,83,0,2.06,BR,1703904066
406,406,porbandar,21.6422,69.6093,22.16,50,0,3.22,IN,1703904351


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df=best_city_df[["City","Country","Lat","Lng","Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name']=""

# Display sample data
hotel_df



,City,Country,Lat,Lng,Humidity,Hotel Name
86,saint croix,VI,17.7397,-64.7388,77,
142,bubaque,GW,11.2833,-15.8333,73,
169,cabo san lucas,MX,22.8909,-109.9124,65,
217,freetown,SL,8.4840,-13.2299,82,
257,lazaro cardenas,MX,17.9583,-102.2000,76,
260,acapulco de juarez,MX,16.8634,-99.8901,73,
345,praya,CV,14.9215,-23.5087,35,
364,salaga,GH,8.5508,-0.5188,18,
385,natal,BR,-5.7950,-35.2094,83,
406,porbandar,IN,21.6422,69.6093,50,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey":geoapify_key,
    "categories": "accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index, 'Lat']
    longitude = hotel_df.loc[index, 'Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places?"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
saint croix - nearest hotel: The Buccaneer
bubaque - nearest hotel: Chez Julio
cabo san lucas - nearest hotel: Comfort Rooms
freetown - nearest hotel: Formal Leona hotel
lazaro cardenas - nearest hotel: Hotel Sol del Pacífico
acapulco de juarez - nearest hotel: Hotel del Valle
praya - nearest hotel: Casa Sodadi
salaga - nearest hotel: No hotel found
natal - nearest hotel: Natal Palace Hotel
porbandar - nearest hotel: Toran Tourist Bungalow
otra banda - nearest hotel: Hotel Don Carlos
bonthe - nearest hotel: No hotel found
ciudad lazaro cardenas - nearest hotel: Hotel Sol del Pacífico
dwarka - nearest hotel: The Dwarika Hotel
mankono - nearest hotel: Hôtel Paillotes
axim - nearest hotel: Axim Beach Hotel
mumford - nearest hotel: Blue Diamond Beach Spa Resort


,City,Country,Lat,Lng,Humidity,Hotel Name
86,saint croix,VI,17.7397,-64.7388,77,The Buccaneer
142,bubaque,GW,11.2833,-15.8333,73,Chez Julio
169,cabo san lucas,MX,22.8909,-109.9124,65,Comfort Rooms
217,freetown,SL,8.4840,-13.2299,82,Formal Leona hotel
257,lazaro cardenas,MX,17.9583,-102.2000,76,Hotel Sol del Pacífico
260,acapulco de juarez,MX,16.8634,-99.8901,73,Hotel del Valle
345,praya,CV,14.9215,-23.5087,35,Casa Sodadi
364,salaga,GH,8.5508,-0.5188,18,No hotel found
385,natal,BR,-5.7950,-35.2094,83,Natal Palace Hotel
406,porbandar,IN,21.6422,69.6093,50,Toran Tourist Bungalow


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [9]:
%%capture --no-display

# Configure the map plot
hotel_df_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City",
    hover_cols = [ "Hotel Name", "Country"]
)

# Display the map
hotel_df_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)